# Chapter 21 - Code and Exercises - Notebook \#1
Code, exercises, and experiments from chapter 21 of "Advances in Financial Machine Learning" by Marcos López de Prado (2018).

In [1]:
# Imports.
import time
import numpy as np
import pandas as pd

from itertools import combinations_with_replacement, product

from mlfinlab.brute_force.combinatorial_opt import *
from mlfinlab.brute_force.comb_opt_example import *

---
## 21.5 AN INTERGER OPTIMIZATION APPROACH
Reframing the asset allocation problem by discretizing the funds available. Consider the number of ways one can allocate $K$ units of capital in $N$ assets, where we assume $K>N$. Essentially we want to find the number of of non-negative integer solutions (though zero is still seems to be a possibility) to $x_1+...+x_N=K$, which has a combinatorial solution $\binom{K+N-1}{N-1}$.

While this bears similarity to a classic integer partitioning problem, note here that order *is* important, since the assets to which capital is allocated is encoded by the order of the permutation. For example, if $K=6$, then partitions $(1,2,3)$ and $(3,2,1)$ are very different ways of allocating units of capital between assets $(x_1, x_2, x_3)$.

Code snippets 21.1, 21.2, and 21.3 are implemented in `mlfinlab.brute_force.combinatorial_opt`, which are algorithms to generate the set of all partitions, $p^{N,K} = \{ \{ p_i \}_{i=1,...,N} | p_i \in \mathbb{W}, \sum_{i=1}^{N} p_i = K \}$, where $\mathbb{W}$ are the set of natural numbers including zero (whole numbers).

---
## 21.6 A NUMERICAL EXAMPLE
Code snippets 21.4, 21.5, and 21.6, which contain functions, are implemented in `mlfinlab.brute_force.comb_opt_example`. The cells below employ these functions to execute the numberical example of dynamic portfolio optimization.

#### Step 1 - **Generate the problem's parameters**

In [2]:
size, horizon = 3, 2
params = []

for h in range(horizon):
    X = random_matrix_with_rank(n_samples=1000, n_cols=3, rank=3, sigma=0.0)
    mean_ = gen_mean(size=size)
    cov_ = np.cov(X, rowvar=False)
    c_ = np.random.uniform(size=cov_.shape[0])*np.diag(cov_)**0.5
    params.append({'mean': mean_, 'cov': cov_, 'c': c_})

#### Step 2 - **Compute and evaluate the static solutions**

In [3]:
w_stat = None

for param in params:
    w_ = stat_opt_portf(cov=param['cov'], a=param['mean'])
    if w_stat is None:
        w_stat = w_.copy()
    else:
        w_stat = np.append(w_stat, w_, axis=1)

t_cost_stat = evaluate_t_costs(w_stat, params)
sr_stat = evaluate_sr(params, w_stat, t_cost_stat)
print(f"Static Sharpe Ratio: {sr_stat}")

Static Sharpe Ratio: -0.5969340385024577


#### Step 3 - **Compute and evaluate the dynamic solution**

In [4]:
w_dynamic = dynamic_optimal_portfolio(params)
t_cost_dynamic = evaluate_t_costs(w_dynamic, params)
sr_dynamic = evaluate_sr(params, w_dynamic, t_cost_dynamic)
print(f"Dynamic Sharpe Ratio: {sr_dynamic}")

Dynamic Sharpe Ratio: 0.8714985128888302
